# DVT Explorer
Sonification of star and planet variables from the Transiting Exoplanet Survey Satellite (TESS) stored in Data Validation Time-series (DVT) files. Data: STScI.

Data download:https://archive.stsci.edu/missions/tess/tid/ 

#### ACKNOWLEDGMENT
This research includes data collected with the TESS mission, obtained from the MAST data archive at the Space Telescope Science Institute (STScI). Funding for the TESS mission is provided by the NASA Explorer Program and by the NASA Science Mission Directorate. STScI is operated by the Association of Universities for Research in Astronomy, Inc., under NASA contract NAS 5–26555.


In [1]:
from astropy.io import fits
from astropy.timeseries import TimeSeries
from astropy.timeseries import BoxLeastSquares
from astropy.stats import sigma_clipped_stats
from astropy.timeseries import aggregate_downsample
from astropy import units as u

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import FunctionTransformer

import os
from pathlib import Path

import time
from pythonosc import udp_client

In [2]:
from astroquery.simbad import Simbad                                                            
from astropy.coordinates import SkyCoord
import astropy.units as u

In [3]:
from urllib.parse import urlencode
import aplpy

In [4]:
root = '## YOUR FOLDER PATH TO THE DOWNLOADED SPECTRA ##'

In [5]:
for path, subdirs, files in os.walk(root):
    for name in files:
        print([os.path.join(path, name)])

['/Volumes/WORK/DOCTORADO_UPM/Library/TESS_DVT_files/s0001/tess2018206190142-s0001-s0001-0000000177179102-00106_dvt.fits']
['/Volumes/WORK/DOCTORADO_UPM/Library/TESS_DVT_files/s0001/tess2018206190142-s0001-s0001-0000000294754396-00106_dvt.fits']
['/Volumes/WORK/DOCTORADO_UPM/Library/TESS_DVT_files/s0001/tess2018206190142-s0001-s0001-0000000080081010-00366_dvt.fits']
['/Volumes/WORK/DOCTORADO_UPM/Library/TESS_DVT_files/s0001/tess2018206190142-s0001-s0001-0000000238895369-00366_dvt.fits']
['/Volumes/WORK/DOCTORADO_UPM/Library/TESS_DVT_files/s0001/tess2018206190142-s0001-s0001-0000000206609630-00106_dvt.fits']
['/Volumes/WORK/DOCTORADO_UPM/Library/TESS_DVT_files/s0001/tess2018206190142-s0001-s0001-0000000237910557-00106_dvt.fits']
['/Volumes/WORK/DOCTORADO_UPM/Library/TESS_DVT_files/s0001/tess2018206190142-s0001-s0001-0000000029861760-00106_dvt.fits']
['/Volumes/WORK/DOCTORADO_UPM/Library/TESS_DVT_files/s0001/tess2018206190142-s0001-s0001-0000000092862191-00106_dvt.fits']
['/Volumes/WORK/

In [6]:
len(files)

100

In [7]:
file = [os.path.join(path, name)]
str = " " 
Ffile = (str.join(file))  

route = Path(Ffile)

Fname = route.with_suffix('')

## Exploring 25 curves

In [8]:
sample = files[0:25]
sample

['tess2018206190142-s0001-s0001-0000000177179102-00106_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000294754396-00106_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000080081010-00366_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000238895369-00366_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000206609630-00106_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000237910557-00106_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000029861760-00106_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000092862191-00106_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000219338901-00106_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000270356871-00106_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000029475080-00106_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000044623916-00106_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000091448148-00106_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000234994474-00366_dvt.fits',
 'tess2018206190142-s0001-s0001-0000000350443417-00106_dvt.fits',
 'tess2018

In [9]:
len(sample)

25

In [10]:
curves=0
i = 0
figure = 0
for path, subdirs, files in os.walk(root):
    for name in sample: 
        curves=curves+1
        file = [os.path.join(path, name)]
        str = " " 
        Ffile = (str.join(file))
        route = Path(name)
        Fname = route.with_suffix('')
        Fpng = route.with_suffix('.png')

#----------------------------------------------        
        dvt_file = Ffile
        
        with fits.open(dvt_file, mode="readonly") as hdulist:
    
            Teff = hdulist[0].header['TEFF']
            Metal = hdulist[0].header['MH']
            Radius = hdulist[0].header['RADIUS']
            Logg = hdulist[0].header['LOGG']
            Sec = int(hdulist[0].header['SECTOR'])
            Tess = hdulist[0].header['TESSMAG']
        
    
            if Teff == None:
                Teff = 0
            if Metal == None:
                Metal = 0
            if Radius == None:
                Radius = 0
            if Logg == None:
                Logg = 0
            if Tess == None:
                Tess = 0
        
            Period = hdulist[1].header['TPERIOD'] 
            Freq = (1/Period)*10000

            Duration = hdulist[1].header['TDUR']
            Depth = hdulist[1].header['TDEPTH']
            
            times = hdulist[1].data['TIME']
            phases = hdulist[1].data['PHASE']
            fluxes_init = hdulist[1].data['LC_INIT']
            model_fluxes_init = hdulist[1].data['MODEL_INIT']
        
        if Duration >= 8:
            figure = 5
        if (Duration >= 4) and (Duration < 8):
            figure = 4
        if (Duration > 2) and (Duration <= 4):
            figure = 3
        if Duration <= (2):
            figure = 2
        
        #---------------------------Lightcurve
        sort_indexes = np.argsort(phases)

        fig, ax = plt.subplots(figsize=(12,4))

        ax.plot(phases[sort_indexes], fluxes_init[sort_indexes], 'go', markersize=1)

        ax.plot(phases[sort_indexes], model_fluxes_init[sort_indexes], '-w')

        fig.suptitle("{id} - {object}".format(id= curves, object= hdulist[0].header['OBJECT']), color = 'w')
        
        ax.set_ylabel("Flux (relative)")
        ax.set_xlabel("Orbital Phase")
        ax.xaxis.label.set_color('w')
        ax.yaxis.label.set_color('w')
        ax.tick_params(axis='x', colors='w')
        ax.tick_params(axis='y', colors='w')
        ax.spines['bottom'].set_color('w')
        ax.spines['top'].set_color('w') 
        ax.spines['right'].set_color('w')
        ax.spines['left'].set_color('w')

        plt.savefig('DVT.png', transparent=True)
        
        plt.close(fig)
        
        #------------------------------StarPlot
        ra_coord = hdulist[0].header['RA_OBJ']
        dec_coord = hdulist[0].header['DEC_OBJ']
        coord = SkyCoord(ra_coord, dec_coord, unit=(u.deg, u.deg), frame='fk5') 
        query_results = Simbad.query_region(coord)   
        object_main_id = query_results[0]['MAIN_ID'].decode('ascii')
        object_coords = SkyCoord(ra=query_results['RA'], dec=query_results['DEC'], unit=(u.hourangle, u.deg), frame='icrs')
    
        query_params = { 
         'hips': 'DSS', 
         'object': object_main_id, 
         'ra': object_coords[0].ra.value, 
         'dec': object_coords[0].dec.value, 
         'fov': (3 * u.arcmin).to(u.deg).value, 
         'width': 800, 
         'height': 350 
        }                                                                                               
        url = f'http://alasky.u-strasbg.fr/hips-image-services/hips2fits?{urlencode(query_params)}' 
        hdul = fits.open(url)
        Plot = plt.figure()
        gc = aplpy.FITSFigure(hdul, figure=Plot)                                                                     
        gc.show_grayscale() 

        gc.show_markers(object_coords.ra, object_coords.dec, edgecolor='green', marker='s', s=50**2)         
        gc.save('plot.png')
        
        gc.refresh() 
        plt.close(Plot)

       #---------------Sending
        print(curves, hdulist[0].header['OBJECT'])
        client_s = udp_client.SimpleUDPClient("127.0.0.1", 9989)
        client_s.send_message("/s", 1)
        
        client9 = udp_client.SimpleUDPClient("127.0.0.1", 9999) 
        client9.send_message("/Sec", Sec)
        print("Sector :", Sec)
        
        client10 = udp_client.SimpleUDPClient("127.0.0.1", 9980) 
        client10.send_message("/Tess", Tess)
        print("Tess Magnitude :", Tess)

        client0 = udp_client.SimpleUDPClient("127.0.0.1", 9990)   
        client0.send_message("/Teff", Teff)
        print("Teff =", Teff)

        client1 = udp_client.SimpleUDPClient("127.0.0.1", 9991) 
        client1.send_message("/Metal", Metal)
        print("Metallicity =", Metal)

        client2 = udp_client.SimpleUDPClient("127.0.0.1", 9992) 
        client2.send_message("/Radius", Radius)
        print("Radius =", Radius)

        client3 = udp_client.SimpleUDPClient("127.0.0.1", 9993) #Surface gravity
        client3.send_message("/Logg", Logg)
        print("Log(g) =", Logg)
    
        client4 = udp_client.SimpleUDPClient("127.0.0.1", 9994) #Duration
        client4.send_message("/Dur", Duration)
        print("Transit duration =", Duration)
        
        client5 = udp_client.SimpleUDPClient("127.0.0.1", 9995) #Amplitude
        client5.send_message("/Amp", Depth)
        print("Transit depth =", Depth)
        
        client6 = udp_client.SimpleUDPClient("127.0.0.1", 9996) 
        client6.send_message("/Freq", Freq)
        print("Transit period =", Period)
        print("Equivalent frequency =", Freq)
        
        client7 = udp_client.SimpleUDPClient("127.0.0.1", 9997) 
        client7.send_message("/Ra", ra_coord)
        print("RA =", ra_coord)
        
        client8 = udp_client.SimpleUDPClient("127.0.0.1", 9998) 
        client8.send_message("/Dec", dec_coord)
        print("DEC =", dec_coord)
        
        
        print("--------------------------")

        i += 1
        time.sleep(figure)
        client_s.send_message("/s", 0)
            
print ("Files represented:",curves, "curves");

#------------STOP
client_s.send_message("/s", 0)
client0.send_message("/Teff", 0)
client1.send_message("/Metal", 0)
client2.send_message("/Radius", 0)
client3.send_message("/Logg",0)
client4.send_message("/Dur", 0)
client5.send_message("/Amp", 0)
client6.send_message("/Freq", 0)
client7.send_message("/Ra", 0)
client8.send_message("/Dec", 0)



INFO: Auto-setting vmin to  7.612e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.536e+04 [aplpy.core]
1 TIC 177179102
Sector : 1
Tess Magnitude : 4.12900019
Teff = 4275.0
Metallicity = 0
Radius = 21.04870033
Log(g) = 1.61693001
Transit duration = 1.0842771305418006
Transit depth = 522.8187340688265
Transit period = 6.855994465873376
Equivalent frequency = 1458.5776067609636
RA = 104.960638
DEC = -67.916359
--------------------------
INFO: Auto-setting vmin to  1.840e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.437e+04 [aplpy.core]
2 TIC 294754396
Sector : 1
Tess Magnitude : 10.67000008
Teff = 3736.0
Metallicity = 0
Radius = 0
Log(g) = 0
Transit duration = 2.399591153942146
Transit depth = 2252.082845732377
Transit period = 22.35891416388731
Equivalent frequency = 447.24891051066163
RA = 71.004173
DEC = -66.401001
--------------------------
INFO: Auto-setting vmin to  2.171e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.641e+04 [aplpy.core]
3 TIC 80081010
Sector : 1
Tess Magnitude : 4

INFO: Auto-setting vmin to  1.624e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.542e+04 [aplpy.core]
20 TIC 270356871
Sector : 1
Tess Magnitude : 8.57586956
Teff = 5964.0
Metallicity = 0
Radius = 1.83414996
Log(g) = 3.94982004
Transit duration = 1.7440716683796356
Transit depth = 3482.066912153904
Transit period = 16.447188649032505
Equivalent frequency = 608.006645597042
RA = 335.171307957536
DEC = -28.2325374132778
--------------------------
INFO: Auto-setting vmin to  2.211e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.222e+04 [aplpy.core]
21 TIC 44623916
Sector : 1
Tess Magnitude : 10.33329964
Teff = 6723.0
Metallicity = 0
Radius = 1.97592998
Log(g) = 4.00186014
Transit duration = 2.4999999999999996
Transit depth = 337891.54127256654
Transit period = 0.9444150328636169
Equivalent frequency = 10588.565039756308
RA = 341.219797980496
DEC = -42.0126577806359
--------------------------
INFO: Auto-setting vmin to  1.553e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.476e+04 [aplpy.cor

In [1]:
import session_info
session_info.show()